# Create layout based on cluster information
* Load network in GraphML
* Convert string group information to integers
* Apply

In [1]:
from graph_tool.all import *

ORIGINAL = "nest2.graphml"
FILTERED = "nest-filtered.graphml" # Score > 0.6

In [2]:
nest = load_graph(ORIGINAL)

In [3]:
members = nest.vertex_properties["layoutMember"]

value_set = set()
for member in members:
    value_set.add(member)

v2idx = {}
idx = 1
for entry in value_set:
    v2idx[entry] = idx
    idx += 1

# Group names to integers
print(v2idx)

{'Group:NEST:29': 1, 'Group:NEST:6': 2, 'Group:NEST:20': 3, 'Group:NEST:13': 4, 'Group:NEST:4': 5, 'Group:NEST:14': 6, 'Group:NEST:18': 7, 'Group:NEST:277': 8, 'Group:NEST:362': 9, 'Group:NEST:5': 10, 'Group:NEST:3': 11, 'Group:NEST:66': 12, 'Group:NEST:11': 13, 'Group:NEST:17': 14, 'Group:NEST:43': 15, 'Group:NEST:12': 16, 'Group:NEST:10': 17, 'Group:NEST:28': 18, 'Group:NEST:217': 19, 'Group:NEST:239': 20, 'Group:NEST:33': 21, 'Group:NEST:145': 22, 'Group:NEST:24': 23, 'Group:NEST:35': 24, 'Group:NEST:15': 25, 'Group:NEST:32': 26, 'Group:NEST:26': 27, 'Group:NEST:238': 28, 'Group:NEST:260': 29, 'Group:NEST:237': 30, 'Group:NEST:19': 31, 'Group:NEST:97': 32, 'Group:NEST:16': 33, 'Group:NEST:325': 34, 'Group:NEST:7': 35, 'Group:NEST:108': 36, 'Group:NEST:8': 37, 'Group:NEST:116': 38, 'Group:NEST:9': 39, 'Group:NEST:2': 40, 'Group:NEST:1': 41, 'Group:NEST:25': 42, 'Group:NEST:23': 43}


In [4]:
# New attribute to show membership as an integer
groups = nest.new_vertex_property("int32_t")
nodes = nest.vertices()
for node in nodes:
    groups[node] = v2idx[members[node]]
    
# Name it "layoutIndex"
nest.vp.layoutIndex = groups

# Edge scores to be used for layout
score = nest.edge_properties["Score"]

In [5]:
# pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=1000, eweight=score, C=0.5, multilevel=True)

# Dense cluster
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=10000, eweight=score, C=0.5, multilevel=True)

# Clusters are closer to each other
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5, multilevel=True)

pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5)


# graph_draw(nest, pos=pos1, vertex_fill_color=groups, edge_color=[0.7, 0.7,0.7, 0.3], output="graph-draw-sfdp-all.pdf")

In [6]:
# Adjust position (For Cytoscape Desktop)
x = nest.new_vertex_property("double")
y = nest.new_vertex_property("double")

# Original positions are too dense for CYtoscape.
SCALING = 100

vs = nest.vertices()
for v in vs:
    p = pos1[v]
    x[v] = p[0] * SCALING
    y[v] = p[1] * SCALING

nest.vp.xpos = x
nest.vp.ypos = y

In [7]:
# list(nest.vertex_properties.keys())
nest.save("export-scaled-full.graphml", fmt="graphml")